In [30]:
#Format Data Correctly
import pandas as pd
import json

# Load JSON data into a pandas DataFrame
with open('data/ASDiv_clean.json', 'r') as json_file:
    data = json.load(json_file)
df = pd.DataFrame(data)

# Specify the columns to remove
columns_to_remove = ['Body', 'Question', 'Solution-Type', 'Answer', 'Formula', 'Grade']

# Remove the specified columns
df = df.drop(columns=columns_to_remove)

# Add a new column with empty values
df['input'] = ""

# Save the modified data back to a JSON file
output_data = df.to_dict(orient='records')
with open('ASDiv_clean_formatted.json', 'w') as json_file:
    json.dump(output_data, json_file)


In [29]:
#Test data loading functions
data_path = 'data/ASDiv_clean_formatted.json'
import os
import json
from datasets import load_dataset, Dataset
def local_dataset(dataset_name):
    if dataset_name.endswith('.json'):
        # Preprocess JSON file to ensure consistent data types
        preprocess_json(dataset_name)
        full_dataset = Dataset.from_json(path_or_paths=[dataset_name])
    elif dataset_name.endswith('.jsonl'):
        full_dataset = Dataset.from_json(filename=dataset_name, format='jsonlines')
    elif dataset_name.endswith('.csv'):
        full_dataset = Dataset.from_pandas(pd.read_csv(dataset_name))
    elif dataset_name.endswith('.tsv'):
        full_dataset = Dataset.from_pandas(pd.read_csv(dataset_name, delimiter='\t'))
    else:
        raise ValueError(f"Unsupported dataset format: {dataset_name}")

    split_dataset = full_dataset.train_test_split(test_size=0.1)
    return split_dataset

def load_data(dataset_name):
        if dataset_name == 'alpaca':
            return load_dataset("tatsu-lab/alpaca")
        elif dataset_name == 'alpaca-clean':
            return load_dataset("yahma/alpaca-cleaned")
        elif dataset_name == 'chip2':
            return load_dataset("laion/OIG", data_files='unified_chip2.jsonl')
        elif dataset_name == 'self-instruct':
            return load_dataset("yizhongw/self_instruct", name='self_instruct')
        elif dataset_name == 'hh-rlhf':
            return load_dataset("Anthropic/hh-rlhf")
        elif dataset_name == 'longform':
            return load_dataset("akoksal/LongForm")
        elif dataset_name == 'oasst1':
            return load_dataset("timdettmers/openassistant-guanaco")
        elif dataset_name == 'vicuna':
            raise NotImplementedError("Vicuna data was not released.")
        else:
            if os.path.exists(dataset_name):
                try:
                    dataset_format =  "input-output"
                    full_dataset = local_dataset(dataset_name)
                    return full_dataset
                except:
                    raise ValueError(f"Error loading dataset from {dataset_name}")
            else:
                raise NotImplementedError(f"Dataset {dataset_name} not implemented yet.")

def format_dataset(dataset, dataset_format):
    if (
        dataset_format == 'alpaca' or dataset_format == 'alpaca-clean' or
        (dataset_format is None and args.dataset in ['alpaca', 'alpaca-clean'])
     ):
           dataset = dataset.map(extract_alpaca_dataset, remove_columns=['instruction'])
    elif dataset_format == 'chip2' or (dataset_format is None and args.dataset == 'chip2'):
        dataset = dataset.map(lambda x: {
            'input': x['text'].split('\n<bot>: ')[0].replace('<human>: ', ''),
            'output': x['text'].split('\n<bot>: ')[1],
          })
    elif dataset_format == 'self-instruct' or (dataset_format is None and args.dataset == 'self-instruct'):
         for old, new in [["prompt", "input"], ["completion", "output"]]:
            dataset = dataset.rename_column(old, new)
    elif dataset_format == 'hh-rlhf' or (dataset_format is None and args.dataset == 'hh-rlhf'):
         dataset = dataset.map(lambda x: {
            'input': '',
            'output': x['chosen']
          })
    elif dataset_format == 'oasst1' or (dataset_format is None and args.dataset == 'oasst1'):
         dataset = dataset.map(lambda x: {
            'input': '',
            'output': x['text'],
         })
    elif dataset_format == 'input-output':
        # leave as is
        pass
    # Remove unused columns.
    dataset = dataset.remove_columns(
        [col for col in dataset.column_names['train'] if col not in ['input', 'output']]
    )
    return dataset

 # Load dataset.
dataset = load_data(data_path)
dataset = format_dataset(dataset, 'input-output')

Extracting data files:   0%|          | 0/1 [00:00<?, ?it/s]

Generating train split: 0 examples [00:00, ? examples/s]

Dataset json downloaded and prepared to /Users/bryan/.cache/huggingface/datasets/json/default-cc95908cddc345aa/0.0.0. Subsequent calls will reuse this data.
